In [ ]:
import $file.common
import spark._
import common._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import spark.implicits._
import spark.sqlContext.implicits._

In [ ]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

# Query 3 y 4:

Se han unificado estas consultas porque comparten gran parte de su construcción y trabajan sobre los mismos conjuntos de datos. En estas consultas se trabaja con los datasets de infecciones diarias reportadas por país y con las tablas de datos demográficos.

### Query 3: Infecciones por número de habitante

El objetivo de esta consulta es determinar el número de infecciones con respecto a la densidad de población por país. Para ello:

    - Se cruzan los datos entre las infecciones diarias y los datos demográficos por país.
    - Se obtiene la tasa de casos entre el número de habitantes para cada día.
    - Se calcula la media de infectados por densidad de población.
    
De esta manera podremos observar la media de infectados diarios por densidad de población.

### Query 4: Tasa diaria de infectados

Esta consulta es similar a la anterior, pero obtiene la tasa de infecciones diarias, de manera que podamos acceder a las fechas más criticas de la enfermedad por país. Para ello:

    - Se cruzan los datos entre las infecciones diarias y los datos demográficos por país.
    - Se obtiene la tasa de casos entre el número de habitantes para cada día.
    - Se ordenan los datos en funcion de la fecha
    
Así podremos observar la evolución de la enfermedad con respecto al calendario


## 1. Consulta con DataSet

In [ ]:
val infectionDS = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/covidworldwide.csv")
    .withColumnRenamed("countriesAndTerritories","Country")
    .as[(String,String,String,String,Double,Double,String,String,String,String,String,String)]

In [ ]:
val populationDS = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/population_by_country_2020.csv")
    .withColumnRenamed("Country (or dependency)","Country")
    .withColumnRenamed("Population (2020)","Population")
    .as[(String,Float,String,Float,Float,Float,Double,String,String,String,String)]

### 1.1 Media diaria de infecciones por número de habitante

In [ ]:
def query_3_DS = 
    infectionDS.join(populationDS, "Country")
            .select($"Country",
                    $"dateRep" as "date",
                    $"cases",
                    $"Land Area (Km\u00b2)",
                   $"cases" / $"Population" as "infection Per Population")
            .groupBy("country")
            .avg("infection Per Population")
            .orderBy(desc("avg(infection Per Population)"))
            .as[(String,Double)]

### 1.2 Porcentaje diario de infectados

In [ ]:
def query_4_DS = 
    infectionDS.join(populationDS, "Country")
            .select($"Country",
                    $"dateRep" as "date",
                    $"cases",
                    $"Land Area (Km\u00b2)",
                   $"cases" / $"Population" as "infection Per Population")
            .orderBy($"date")
            .as[(String,String,Int,String,Double)]

## 2 Consulta con DataFrame

In [ ]:
val dfCovid = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/covidworldwide.csv")

In [ ]:
val dfPopulation = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/population_by_country_2020.csv")
    .withColumnRenamed("Country (or dependency)","Country")
    .withColumnRenamed("Population (2020)","Population")

Modifico los datos de entrada para que el formato fecha se adecue al TimeStamp de Spark

In [ ]:
def dfCovidClean = dfCovid
    .select($"*",$"dateRep",translate($"dateRep","/","-").as("date"))
    .drop("dateRep")

In [ ]:
def dfCovidDate = dfCovidClean
    .select($"*",col("date"),to_date(col("date"),"dd-MM-yyyy").as("to_date"))

### 2.1 Media diaria de infecciones por número de habitante

In [ ]:
def query_3_DF = dfCovid.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"dateRep" as "date",
                $"cases",
                $"Population",
                $"cases" / $"Population" as "infection Per Population")
        .groupBy("country")
        .avg("infection Per Population")
        .orderBy(desc("avg(infection Per Population)"))

### 2.2 Porcentaje diario de infectados

In [ ]:
def query_4_DF =
dfCovidDate.join(dfPopulation, $"country" === $"countriesAndTerritories")
        .select($"country",
                $"to_date",
                $"day",
                $"month",
                $"cases",
                $"Population",
                $"cases" / $"Population" as "infection Per Population")
        .orderBy($"to_date".asc)

## 3. Visualización de rendimiento

In [ ]:
val (x, y) = Seq(
    "DataSet 3" -> runWithOutput(query_3_DS.collect),
    "DataSet 4" -> runWithOutput(query_4_DS.collect),    
    "DataFrame 3" -> runWithOutput(query_3_DS.collect),
    "DataFrame 4" -> runWithOutput(query_4_DS.collect),
).unzip

Bar(x, y).plot()

## 4. Comparativa de rendimiento

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_3_DS.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_4_DS.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_3_DF.collect)

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_4_DF.collect)

## 5. Visualización de datos con plotly

### Media de infecciones por densidad de población

In [ ]:
val (x,y) = query_3_DF.collect.map(r=>(r(0).toString, r(1).toString)).toList.unzip
Bar(x, y).plot()

### Porcentaje diario de infectados en España

In [ ]:
val (x,y) = query_4_DF
    .filter($"country" === "Spain")
    .collect+
    .map(r=>(r(1).toString, r(6).toString))
    .toList
    .unzip
Bar(x, y).plot()

### Comparacion entre paises de crecimiento de la enfermedad

In [ ]:
val y = query_4_DF
    .filter($"country" === "Spain")
    .select($"infection Per Population")
    .collect
    .map(r => r(0).toString.toDouble)
    .toList

val x = query_4_DF
    .filter($"country" === "Spain")
    .select($"to_date")
    .collect
    .toList
    .map(_.toString)

val y1 = query_4_DF
    .filter($"country" === "Italy")
    .select($"infection Per Population")
    .collect
    .map(r => r(0).toString.toDouble)
    .toList

val x1 = query_4_DF
    .filter($"country" === "Italy")
    .select($"to_date")
    .collect
    .toList
    .map(_.toString)

val data = Seq(
        Scatter(x,y).withName("Spain"),
        Scatter(x1,y1,mode = ScatterMode(ScatterMode.Lines),
        line = Line(color = Color.StringColor("#7F7F7F"))).withName("Italy"))
    .map(_.withFill(Fill.ToNextY)
    .withStackgroup("A"))

plot(data)